In [23]:
class merkeltreecommit:
    def __init__(self, mesg_byte = 24 , mesg_len = 2**16 , hash_byte = 32 , leaf_padding_byte = 32 ):
        self.hash_byte = hash_byte
        self.leaf_padding_byte = leaf_padding_byte
        self.mesg_byte = mesg_byte
        assert( 0==(mesg_len&(mesg_len-1)) )   # mesg_len is a power of 2
        self.mesg_len   = mesg_len

    @staticmethod
    def authpath_len( leaf_size ):
        r = 0
        layer_size = leaf_size
        while(layer_size > 1):
            r += 1
            layer_size //= 2
        return r

    def get_proof_size(self):
        r = self.mesg_byte + self.leaf_padding_byte
        r += merkeltreecommit.authpath_len( self.mesg_len ) * self.hash_byte
        return r
    
    def states(self):
        r = vars(self)
        r['authpath_len'] = merkeltreecommit.authpath_len(self.mesg_len)
        r['proof_size']  = self.get_proof_size()
        return r


In [24]:
mc0 = merkeltreecommit()
print( "|auth path|:" , mc0.authpath_len( mc0.mesg_len ) )
print( "size(auth path):" , mc0.get_proof_size() )
print( mc0.states() )

|auth path|: 16
size(auth path): 568
{'hash_byte': 32, 'leaf_padding_byte': 32, 'mesg_byte': 24, 'mesg_len': 65536, 'authpath_len': 16, 'proof_size': 568}


In [5]:
class fri :
    def __init__(self, field_byte = 24 , hash_byte = 32 , poly_len = 2**16 ):
        self.fielf_byte = field_byte
        self.hash_byte  = hash_byte
        assert( 0==(poly_len&(poly_len-1)) )   # poly_len is a power of 2
        self.poly_len   = poly_len

    def get_proof_size(self):
        return 0

In [6]:
pc0 = fri()
print( pc0.get_proof_size() )

0
